# Storage Class Testing Notebook

In [11]:
from Storage import StorageSuite
import numpy as np
import matplotlib.pyplot as plt
import sys
import pymgrid as pgd
import pandas as pd
from IPython.display import display, clear_output
import csv
import ast
import operator as op
from math import e
import gc
from DQNEnv import DQAgent # Neural Net implimentation module
import pickle as pkl # Neural Network storage and loading
from pymgrid.MicrogridGenerator import MicrogridGenerator

HOUR = 4 # 15 Min intervals in an hour
DAY = 96 # Number of 15 Min intervals in day
MONTH = 2_924 # Number of 15 Min intervals in month
YEAR =35_088 # Number of 15 Min intervals in year (non-leap)
ZERO = 10**-5 # Low value for zeroes

############
''' the following is a string -> evaluation parser '''
operators = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
             ast.Div: op.truediv, ast.Pow: op.pow, ast.BitXor: op.xor,
             ast.USub: op.neg}

def eval_expr(expr) -> float :
    if expr != '':
        return eval_(ast.parse(expr, mode='eval').body)
    else:
        return None

def eval_(node) -> float:
    if isinstance(node, ast.Num): # <number>
        return node.n
    elif isinstance(node, ast.BinOp): # <left> <operator> <right>
        return operators[type(node.op)](float(eval_(node.left)), float(eval_(node.right)))
    elif isinstance(node, ast.UnaryOp): # <operator> <operand> e.g., -1
        return operators[type(node.op)](eval_(node.operand))
    else:
        raise TypeError(node)
############


Create Environment: Use StorageSuite

In [2]:
env = StorageSuite(r'C:\Users\thesu\Desktop\Research 2022 Microgrid\Research code repository\Micro-Grid-ML-algo\data\energy_storage_devices_v6.csv', load = 6E5) # load=np.random.randint(1E5,6E5))
li,flow,fly = env.unpack()
print(f'Li: {li.cap}, FLow: {flow.cap}, Fly: {fly.cap}')

Li: 200000.0, FLow: 200000.0, Fly: 200000.0


In [3]:
# def cap_function(params: list):
#         cap_cost_formula = env.get_total_capital_cost_formula().replace("L", str(params[0]/1000)).replace("F", str(params[1]/1000)).replace("W", str(params[2]/1000))
#         return cap_cost_formula 
# params = [li.cap,flow.cap,fly.cap]
# print(cap_function(params=params))
# print(env.get_capital_cost())

In [4]:
env.modify_ss([2E5,2E5,2E5])
li,flow,fly = env.unpack()
print(f'Li: {li.cap}, FLow: {flow.cap}, Fly: {fly.cap}')


Li: 200000.0, FLow: 200000.0, Fly: 200000.0


In [8]:
def test_grid(env: object, horizon: int, load_path: str) -> float:
    ''' Manages the grid based on a trained neural net model '''
    env.set_horizon(horizon = horizon) # Sets the Horizon

    with open(load_path, 'rb') as f:  # Loads agent from desired path
        agent = pkl.load(f)

    state = env.reset()                                                       
    score = 0                                                                   
    done = 0

    #main testing loop                                                            
    while not done:                                                                                         
        action_select = agent.choose_action(state)
        action = env.actions_agent(action = action_select) 
        new_state,reward, done, = env.run(action)                             
        score+=reward                                                                                                                   
        state = new_state
        value_print="\rProgress " + str(round(((env._tracking_timestep)*1000)/(env.horizon*10),1)) +" %"
        sys.stdout.write(value_print)
        sys.stdout.flush()                                                       
    env.reset()                                                                    
    return score

In [9]:
mg1 = MicrogridGenerator(storage_suite_list=[env])
mg1.generate_microgrid(verbose=False, interpolate=True)
mg_env = mg1.microgrids[0]
li,flow,fly = env.unpack()
print(f'Li: {li.cap}, FLow: {flow.cap}, Fly: {fly.cap}')

Li: 200000.0, FLow: 200000.0, Fly: 200000.0


In [12]:
score1 = test_grid(env = mg_env, horizon=YEAR, load_path=r"C:\Users\thesu\Desktop\Design\Agents\Trained Agent Object.pkl")

Progress 99.9 %

In [13]:
print(score1)

35538789.802956


In [29]:
env.modify_ss([1E4,1E4,1E4])
mg1 = MicrogridGenerator(storage_suite_list=[env])
mg1.generate_microgrid(verbose=False, interpolate=True)
mg_env = mg1.microgrids[0]
li,flow,fly = env.unpack()
print(f'Li: {li.cap}, FLow: {flow.cap}, Fly: {fly.cap}')

Li: 10000.0, FLow: 10000.0, Fly: 10000.0


In [30]:
score3 = test_grid(env = mg_env, horizon=YEAR, load_path=r"C:\Users\thesu\Desktop\Design\Agents\Trained Agent Object.pkl")

Progress 99.9 %

In [31]:
print(score3-score1)

13825.71586497128
